In [1]:
%load_ext autoreload
%autoreload 2

import xbitdiff

In [2]:
import xarray as xr
import zarr

create the dataset

In [6]:
import numpy as np
import pandas as pd

np.random.seed(0)
xy_dim = 50
temperature = 15 + 8 * np.random.randn(xy_dim, xy_dim, 100)
lon = np.tile(np.linspace(-99, -98, num=xy_dim), (xy_dim, 1))
lat = np.tile(np.linspace(42, 43, num=xy_dim), (xy_dim, 1))
time = pd.date_range("2014-09-06", periods=100)
reference_time = pd.Timestamp("2014-09-05")

create the dataset

In [10]:
ds = xr.Dataset(
    data_vars=dict(
        temperature=(["x", "y", "time"], temperature),
    ),
    coords=dict(
        lon=(["x", "y"], lon),
        lat=(["x", "y"], lat),
        time=time,
        reference_time=reference_time,
    ),
    attrs=dict(description="Random temperature data."),
)

and save it as a zarr

In [11]:
import zarr

compressor = zarr.Blosc(cname="zstd", clevel=3, shuffle=2)
encoding = {x: {"compressor": compressor} for x in ds}
ds.to_zarr('source.zarr', encoding=encoding)

Now open, create a new dataset, a diff, and save them back to zarr.

In [12]:
import xbitdiff

ds = xbitdiff.open_dataset('source.zarr', engine='zarr')

# create a new dataset
ds2 = ds + 1

# compute the diff
diff = ds2.bitdiff.diff(ds)

# now save the new dataset and the diff
ds2.to_zarr('source2.zarr', encoding=encoding)
diff.to_zarr('diff.zarr', encoding=encoding)

Finally, check the size of the compressed zarr store.

In [13]:
!du -h --max-depth=0 *.zarr

36K	diff.zarr
1.8M	source.zarr
1.8M	source2.zarr
